# Zero to Singularity: Create, Tune, Deploy and Scale a Deep Neural Network in 90 Minutes

This notebook is part of a masterclass held at IBM Think on 13th of February 2019 in San Fransisco
In this exercise you will train a Keras DeepLearning model running on top of TensorFlow. 

Note: For sake of bringing the training runtime down we've done two things

1) Used a softmax regression model over a Convolutional Neural Network 

2) Trained only for one epoch instead of 20

This leads to approx. 5% less accuracy


Authors

Romeo Kienzler - Chief Data Scientist, IBM Watson IoT

Krishnamurthy Arthanarisamy - Architect, Watson Machine Learning Software Lab, Bangalore


# Prerequisites

Please make sure the currently installed version of Keras and Tensorflow are matching the requirememts, if not, please run the two PIP commands below in order to re-install. Please restart the kernal before proceeding, please re-check if the versions are matching.

In [132]:
import keras
print('Current:\t', keras.__version__)
print('Expected:\t 2.1.3')

Current:	 2.1.3
Expected:	 2.1.3


In [133]:
import tensorflow as tf
print('Current:\t', tf.__version__)
print('Expected:\t 1.5.0')

Current:	 1.5.0
Expected:	 1.5.0


In [ ]:
!pip install keras==2.1.3

In [ ]:
!pip install tensorflow==1.5.0

# 1.0 Train a MNIST digits recognition model
We start with some global parameters and imports

In [134]:
#some learners constantly reported 502 errors in Watson Studio. 
#This is due to the limited resources in the free tier and the heavy resource consumption of Keras.
#This is a workaround to limit resource consumption

from keras import backend as K

K.set_session(K.tf.Session(config=K.tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)))

In [135]:
import keras
from keras.models import Model
from keras.layers import Input, Dense
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.datasets import mnist
from keras.models import Sequential, load_model
from keras.optimizers import RMSprop
from keras.layers import LeakyReLU

from keras import backend as K
import numpy as np

In [137]:
batch_size = 128
num_classes = 10
epochs = 1

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255


# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


# Training a simple model
First we'll train a simple softmax regressor and check what accuracy we get

In [138]:
model = Sequential()
model.add(Dense(512, input_shape=(784,)))
model.add(Dense(num_classes, activation='softmax'))



model.compile(loss='categorical_crossentropy',
        optimizer='rmsprop',
        metrics=['accuracy'])

model.fit(x_train, y_train,
        batch_size=batch_size,
        epochs=epochs,
        verbose=1,
        validation_data=(x_test, y_test))
        
score = model.evaluate(x_test, y_test, verbose=0)

print('\n')
print('Accuracy:',score[1])

Train on 60000 samples, validate on 10000 samples
Epoch 1/1
60000/60000 [==============================] - 14s 237us/step - loss: 0.3924 - acc: 0.8862 - val_loss: 0.3120 - val_acc: 0.9144


Accuracy: 0.9144


In [139]:
#some cleanup from the previous run
!rm ker_*
!rm my_best_model.tgz

You should see an accuracy of approximately 90%. Now lets define a hyper-parameter grid including different activation functions and gradient descent optimizers. We’re optimizing over the grid using grid search (nested for loops) and store each model variant in a file. We then decide for the best one in order to deploy to IBM Watson Machine Learning.

In [140]:
#define parameter grid

activation_funcitons_layer_1 = ['sigmoid','tanh','relu']
opimizers = ['rmsprop','adagrad','adadelta']

#optimize over parameter grid (grid search)

for activation_funciton_layer_1 in activation_funcitons_layer_1:
    for opimizer in opimizers:
        
        model = Sequential()
        model.add(Dense(512, activation = activation_funciton_layer_1, input_shape=(784,)))
        model.add(Dense(num_classes, activation='softmax'))



        model.compile(loss='categorical_crossentropy',
              optimizer=opimizer,
              metrics=['accuracy'])

        model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              verbose=1,
              validation_data=(x_test, y_test))
        
        score = model.evaluate(x_test, y_test, verbose=0)
        save_path = "ker_func_mnist_model_2.%s.%s.%s.h5" % (activation_funciton_layer_1,opimizer,score[1])
        model.save(save_path)

Train on 60000 samples, validate on 10000 samples
Epoch 1/1
60000/60000 [==============================] - 14s 235us/step - loss: 0.4240 - acc: 0.8845 - val_loss: 0.2618 - val_acc: 0.9253
Train on 60000 samples, validate on 10000 samples
Epoch 1/1
60000/60000 [==============================] - 13s 222us/step - loss: 0.4377 - acc: 0.8840 - val_loss: 0.2957 - val_acc: 0.9174
Train on 60000 samples, validate on 10000 samples
Epoch 1/1
60000/60000 [==============================] - 16s 274us/step - loss: 0.5483 - acc: 0.8609 - val_loss: 0.3305 - val_acc: 0.9040
Train on 60000 samples, validate on 10000 samples
Epoch 1/1
60000/60000 [==============================] - 13s 219us/step - loss: 0.3290 - acc: 0.9038 - val_loss: 0.1979 - val_acc: 0.9400
Train on 60000 samples, validate on 10000 samples
Epoch 1/1
60000/60000 [==============================] - 13s 223us/step - loss: 0.3218 - acc: 0.9084 - val_loss: 0.2187 - val_acc: 0.9373
Train on 60000 samples, validate on 10000 samples
Epoch 1/1


# Model evaluation
Let's have a look at all the models and see which hyper parameter configuration was the best one. You should see that relu and rmsprop gives you > 95% of accuracy on the validation set

In [141]:
ls -ltr ker_*

-rw-r----- 1 dsxuser dsxuser 3276568 Feb 12 14:54 ker_func_mnist_model_2.sigmoid.rmsprop.0.9253.h5
-rw-r----- 1 dsxuser dsxuser 3276568 Feb 12 14:54 ker_func_mnist_model_2.sigmoid.adagrad.0.9174.h5
-rw-r----- 1 dsxuser dsxuser 4905136 Feb 12 14:54 ker_func_mnist_model_2.sigmoid.adadelta.0.904.h5
-rw-r----- 1 dsxuser dsxuser 3276568 Feb 12 14:55 ker_func_mnist_model_2.tanh.rmsprop.0.94.h5
-rw-r----- 1 dsxuser dsxuser 3276568 Feb 12 14:55 ker_func_mnist_model_2.tanh.adagrad.0.9373.h5
-rw-r----- 1 dsxuser dsxuser 4905136 Feb 12 14:55 ker_func_mnist_model_2.tanh.adadelta.0.9245.h5
-rw-r----- 1 dsxuser dsxuser 3276568 Feb 12 14:56 ker_func_mnist_model_2.relu.rmsprop.0.9557.h5
-rw-r----- 1 dsxuser dsxuser 3276568 Feb 12 14:56 ker_func_mnist_model_2.relu.adagrad.0.9481.h5
-rw-r----- 1 dsxuser dsxuser 4905136 Feb 12 14:56 ker_func_mnist_model_2.relu.adadelta.0.9517.h5


Now it's time to create a tarball out of your favorite model, please replace the name of your favorite model H5 file with “please-put-me-here”

In [142]:
!tar -zcvf my_best_model.tgz please-put-me-here.h5

ker_func_mnist_model_2.relu.rmsprop.0.9557.h5


## 2.0 Save the trained model to WML Repository

We will use `watson_machine_learning_client` python library to save the trained model to WML Repository, to deploy the saved model and to make predictions using the deployed model.</br>


`watson_machine_learning_client` can be installed using the following `pip` command:

`!pip install watson-machine-learning-client --upgrade`

In [71]:
!pip show watson-machine-learning-client 

Name: watson-machine-learning-client
Version: 1.0.133
Summary: Watson Machine Learning API Client
Home-page: http://wml-api-pyclient.mybluemix.net
Author: IBM
Author-email: lukasz.cmielowski@pl.ibm.com, maria.oleszkiewicz@pl.ibm.com, wojciech.sobala@pl.ibm.com
License: BSD
Location: /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages
Requires: certifi, requests, tabulate, pandas, lomond, ibm-cos-sdk, tqdm, urllib3


In [78]:
mkdir temp_install

mkdir: cannot create directory ‘temp_install’: File exists


In [79]:
#!pip install watson-machine-learning-client --upgrade
!pip install -b ./temp_install --upgrade --index-url https://test.pypi.org/simple/ --extra-index-url https://pypi.python.org/simple/ watson-machine-learning-client==1.0.133
        

Requirement already up-to-date: watson-machine-learning-client==1.0.133 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages
Requirement not upgraded as not directly required: lomond in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from watson-machine-learning-client==1.0.133)
Requirement not upgraded as not directly required: tqdm in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from watson-machine-learning-client==1.0.133)
Requirement not upgraded as not directly required: requests in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from watson-machine-learning-client==1.0.133)
Requirement not upgraded as not directly required: ibm-cos-sdk in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from watson-machine-learning-client==1.0.133)
Requirement not upgraded as not directly required: tabulate in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from watson-machine-learning-client==1.0.133)
Requirement not upgraded as not dire

In [143]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient

In [ ]:
wml_credentials={
  "url": "https://ibm-watson-ml.mybluemix.net",
  "access_key": "Uxormn3AYG5S67ZzJswnXa6XkpFABZZjGm",
  "username": "xx",
  "password": "yy",
  "instance_id": "iiii"
}

In [144]:
wml_credentials={
  "apikey": "MQSS8l6H30IlHf-xDj9PaDqdm0w-yYwPAugDX0T4FNPd",
  "iam_apikey_description": "Auto generated apikey during resource-key operation for Instance - crn:v1:bluemix:public:pm-20:us-south:a/4b5f219cdaee498f9dac672a8966c254:520a7acd-b58d-48bb-95ad-4c7a2f922839::",
  "iam_apikey_name": "auto-generated-apikey-193aed8e-6f01-45c5-81c1-66d196d262f0",
  "iam_role_crn": "crn:v1:bluemix:public:iam::::serviceRole:Manager",
  "iam_serviceid_crn": "crn:v1:bluemix:public:iam-identity::a/4b5f219cdaee498f9dac672a8966c254::serviceid:ServiceId-60ecd3d6-7dbd-4cf0-8bc0-dc061c63c713",
  "instance_id": "520a7acd-b58d-48bb-95ad-4c7a2f922839",
  "password": "",
  "url": "https://us-south.ml.cloud.ibm.com",
  "username": "193aed8e-6f01-45c5-81c1-66d196d262f0"
}

In [145]:
client = WatsonMachineLearningAPIClient(wml_credentials)

In [146]:
import json

In [147]:
model_props = {client.repository.ModelMetaNames.AUTHOR_NAME: "IBM", 
               client.repository.ModelMetaNames.AUTHOR_EMAIL: "ibm@ibm.com", 
               client.repository.ModelMetaNames.NAME: "KK3_clt_keras_mnist",
               client.repository.ModelMetaNames.FRAMEWORK_NAME: "tensorflow",
               client.repository.ModelMetaNames.FRAMEWORK_VERSION: "1.5" ,
               client.repository.ModelMetaNames.FRAMEWORK_LIBRARIES: [{"name": "keras", "version": "2.1.3"}]
              }

In [148]:
published_model = client.repository.store_model(model="my_best_model.tgz", meta_props=model_props)


In [149]:
import json

In [150]:
published_model_uid = client.repository.get_model_uid(published_model)
model_details = client.repository.get_details(published_model_uid)

In [ ]:
### Delete the model from WML Repository
### client.repository.delete(published_model_uid) 

## 3.0 Deploy the Keras model

In [151]:
client.deployments.list()

------------------------------------  -------------------  ------  --------------  ------------------------  --------------
GUID                                  NAME                 TYPE    STATE           CREATED                   FRAMEWORK
5790bd41-d42f-4b73-9695-4fc890c49edd  k1_keras_mnist_clt1  online  DEPLOY_SUCCESS  2019-02-12T13:29:59.382Z  tensorflow-1.5
------------------------------------  -------------------  ------  --------------  ------------------------  --------------


To keep your environment clean, just delete all deployments from previous runs

In [152]:
client.deployments.delete("5790bd41-d42f-4b73-9695-4fc890c49edd")

In [153]:
created_deployment = client.deployments.create(published_model_uid, name="k1_keras_mnist_clt1")



#######################################################################################

Synchronous deployment creation for uid: '033564d8-b797-4c9a-a175-89222c4d5345' started

#######################################################################################


INITIALIZING
DEPLOY_IN_PROGRESS
DEPLOY_SUCCESS


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='033564d8-b797-4c9a-a175-89222c4d5345'
------------------------------------------------------------------------------------------------




## Test the model

In [154]:
#scoring_endpoint = client.deployments.get_scoring_url(created_deployment)
scoring_endpoint = created_deployment['entity']['scoring_url']
print(scoring_endpoint)

https://us-south.ml.cloud.ibm.com/v3/wml_instances/520a7acd-b58d-48bb-95ad-4c7a2f922839/deployments/033564d8-b797-4c9a-a175-89222c4d5345/online


In [155]:
x_score_1 = x_test[23].tolist()
print('The answer should be: ',np.argmax(y_test[23]))
scoring_payload = {'values': [x_score_1]}

The answer should be:  5


In [156]:
predictions = client.deployments.score(scoring_endpoint, scoring_payload)
print('And the answer is!... ',predictions['values'][0][1])

And the answer is!...  5


In [159]:
x_score_1

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.2705882489681244,
 0.5960784554481506,
 0.929411768913269,
 0